In [1]:
import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [2]:
from functools import partial

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

training = tf.placeholder_with_default(False, shape=(), name="training")
bn_layer = partial(tf.layers.batch_normalization, training=training, momentum=.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = bn_layer(hidden1)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = bn_layer(hidden2)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=.9)

In [3]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [4]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [5]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../chapter_10/data/mnist")

n_epochs = 40
batch_size = 50

Extracting ../chapter_10/data/mnist\train-images-idx3-ubyte.gz
Extracting ../chapter_10/data/mnist\train-labels-idx1-ubyte.gz
Extracting ../chapter_10/data/mnist\t10k-images-idx3-ubyte.gz
Extracting ../chapter_10/data/mnist\t10k-labels-idx1-ubyte.gz


In [9]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={training:True, X:X_batch, y:y_batch})
            
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y:mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Test accuracy: 0.922
1 Test accuracy: 0.944
2 Test accuracy: 0.9553
3 Test accuracy: 0.9608
4 Test accuracy: 0.9659
5 Test accuracy: 0.9663
6 Test accuracy: 0.9687
7 Test accuracy: 0.9709
8 Test accuracy: 0.9722
9 Test accuracy: 0.9719
10 Test accuracy: 0.9729
11 Test accuracy: 0.9759
12 Test accuracy: 0.9746
13 Test accuracy: 0.9766
14 Test accuracy: 0.9755
15 Test accuracy: 0.9766
16 Test accuracy: 0.9761
17 Test accuracy: 0.9768
18 Test accuracy: 0.9757
19 Test accuracy: 0.9772
20 Test accuracy: 0.9772
21 Test accuracy: 0.9769
22 Test accuracy: 0.979
23 Test accuracy: 0.9772
24 Test accuracy: 0.9782
25 Test accuracy: 0.9782
26 Test accuracy: 0.9777
27 Test accuracy: 0.9782
28 Test accuracy: 0.9793
29 Test accuracy: 0.9787
30 Test accuracy: 0.9786
31 Test accuracy: 0.9775
32 Test accuracy: 0.9789
33 Test accuracy: 0.9781
34 Test accuracy: 0.9778
35 Test accuracy: 0.9791
36 Test accuracy: 0.9797
37 Test accuracy: 0.9782
38 Test accuracy: 0.9796
39 Test accuracy: 0.9796


In [14]:
imported_meta = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

In [15]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
training/input
training
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
hidden1/MatMul
hidden1/BiasAdd
batch_normalization/beta/Initializer/zeros
batch_normalization/beta
batch_normalization/beta/Assign
batch_normalization/beta/read
batch_normalization/gamma/Initializer/ones
batch_normalization/gamma
batch_normalization/gamma/Assign
batch_normalization/gamma/read
batch_normalization/moving_mean/Initializer/zeros
batch_normalization/moving_mean
batch_normalization/moving_mean/Assign
batch_normalization/moving_mean/read
batch_normalization/moving_variance/Initializer/one

train/gradients/batch_normalization_2/moments/Mean_1_grad/Prod_1_1
train/gradients/batch_normalization_2/moments/Mean_1_grad/Maximum_1/y_1
train/gradients/batch_normalization_2/moments/Mean_1_grad/Maximum_1_1
train/gradients/batch_normalization_2/moments/Mean_1_grad/floordiv_1_1
train/gradients/batch_normalization_2/moments/Mean_1_grad/Cast_1
train/gradients/batch_normalization_2/moments/Mean_1_grad/truediv_1
train/gradients/batch_normalization_2/moments/Square_grad/mul/x_1
train/gradients/batch_normalization_2/moments/Square_grad/mul_2
train/gradients/batch_normalization_2/moments/Square_grad/mul_1_1
train/gradients/batch_normalization_2/moments/SquaredDifference_grad/Shape_2
train/gradients/batch_normalization_2/moments/SquaredDifference_grad/Shape_1_1
train/gradients/batch_normalization_2/moments/SquaredDifference_grad/BroadcastGradientArgs_1
train/gradients/batch_normalization_2/moments/SquaredDifference_grad/scalar_1
train/gradients/batch_normalization_2/moments/SquaredDifference_